<a href="https://colab.research.google.com/github/deiveleal/multi_modal_machine_learning/blob/main/PrimeiroModelo/MultiModalAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Multimodal Algorithm

Data &#8594; Pre-process &#8594; Separate Training &#8594; Fusion &#8594; Metric

### Hello world (Iris)

In [ ]:
# import os

# def rename_files_in_directory(directory_path):
#     """
#     Renomeia todos os arquivos em um diretório para o nome da pasta seguido de um número crescente.
#     Substitui "-" por "_" no nome da pasta.

#     Args:
#         directory_path: Caminho para o diretório.
#     """
#     # Obtém o nome da pasta e substitui "-" por "_"
#     folder_name = os.path.basename(directory_path).replace("-", "_")
#     # Lista todos os arquivos no diretório
#     filenames = os.listdir(directory_path)

#     # Para cada arquivo no diretório
#     for i, filename in enumerate(filenames):
#         # Constrói o novo nome do arquivo
#         new_filename = f"{folder_name}_{i+1}.jpg"
#         # Constrói o caminho completo para o arquivo antigo e o novo
#         old_file_path = os.path.join(directory_path, filename)
#         new_file_path = os.path.join(directory_path, new_filename)
#         # Renomeia o arquivo
#         os.rename(old_file_path, new_file_path)

# # Uso da função
# rename_files_in_directory('sample/iris/iris-virginica')

### Processa imagem (cria um vetor com a imagem)

In [1]:
import os

import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
from PIL import Image
import cv2
import pandas as pd

# Alterar as opções de impressão
# np.set_printoptions(threshold=np.inf)

In [2]:
def vetorizar_imagem(caminho_imagem):
    # Ler a imagem
    imagem = cv2.imread(caminho_imagem)

    # Converter a imagem em um vetor unidimensional
    vetor_imagem = imagem.flatten()

    return vetor_imagem

In [3]:
def vetorizar_imagens(diretorio):
    vetor_imagens = {}
    for root, dirs, files in os.walk(diretorio):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                caminho_imagem = os.path.join(root, filename)
                imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
                vetor_imagem = imagem.flatten()
                file_name = filename.split('.')[0]
                vetor_imagens[file_name] = np.array(vetor_imagem)
    return vetor_imagens

In [4]:
# Use a função para vetorizar todas as imagens em um diretório
# images_vectorized = {}
directory = 'sample/iris/test'
# images_vectorized = vetorizar_imagens(directory)
# Exemplo de uso
vetor_imagens = vetorizar_imagens(directory)

In [5]:
vetor_imagens

{'iris_versicolor_269': array([66, 69, 73, ...,  8,  8,  8], dtype=uint8),
 'iris_setosa_65': array([184, 183, 185, ..., 153, 134, 157], dtype=uint8),
 'iris_setosa_66': array([209, 131,  89, ...,  76,  97,  53], dtype=uint8),
 'iris_virginica_71': array([86, 86, 87, ..., 80, 79, 78], dtype=uint8),
 'iris_versicolor_233': array([ 76,  79,  82, ..., 124, 126, 128], dtype=uint8),
 'iris_versicolor_223': array([ 5,  8,  4, ..., 38, 39, 39], dtype=uint8),
 'iris_versicolor_240': array([83, 47, 37, ..., 68, 67, 88], dtype=uint8),
 'iris_versicolor_261': array([130, 131, 133, ...,  96,  85,  87], dtype=uint8),
 'iris_virginica_81': array([ 14,  14,  14, ..., 148, 143, 136], dtype=uint8),
 'iris_versicolor_256': array([ 88,  88,  86, ..., 126, 133, 137], dtype=uint8),
 'iris_versicolor_236': array([69, 54, 46, ..., 82, 85, 86], dtype=uint8),
 'iris_setosa_67': array([140,  90,  58, ...,  99,  45,  45], dtype=uint8),
 'iris_versicolor_260': array([ 3,  0, 18, ..., 22,  2,  1], dtype=uint8),
 '

In [6]:
def extrair_cor_petalas(caminho_imagem):
    # Carregar a imagem
    imagem = Image.open(caminho_imagem)
    imagem = imagem.resize((150,150)) # Redimensionar a imagem para reduzir o tempo de processamento
    imagem_np = np.array(imagem)

    # Redimensionar a imagem para ser uma lista de pixels
    pixels = imagem_np.reshape(-1, 3)

    # Executar o algoritmo k-means para encontrar as cores mais comuns
    kmeans = KMeans(n_clusters=5)
    kmeans.fit(pixels)

    # Encontrar a cor mais comum (ou seja, o centro do cluster mais comum)
    cor_dominante = kmeans.cluster_centers_[kmeans.labels_[0]]

    return cor_dominante

In [7]:
def extrair_cores_pasta(caminho_pasta):
    # Inicializar um dicionário para armazenar as cores dominantes
    cores_dominantes = {}

    # Listar todos os arquivos na pasta
    arquivos = os.listdir(caminho_pasta)

    # Para cada arquivo na pasta, extrair a cor dominante das pétalas
    for arquivo in arquivos:
        # Ignorar arquivos que não são imagens
        if not (arquivo.endswith('.jpg') or arquivo.endswith('.png')):
            continue

        # Construir o caminho completo para o arquivo
        caminho_arquivo = os.path.join(caminho_pasta, arquivo)

        # Extrair a cor dominante das pétalas
        cor_dominante = extrair_cor_petalas(caminho_arquivo)

        # Adicionar a cor dominante ao dicionário
        cores_dominantes[arquivo.split('.')[0]] = cor_dominante

    return cores_dominantes


In [8]:
# Exemplo de uso
cores_dominantes = extrair_cores_pasta('sample/iris/test')
for nome_arquivo, cor_dominante in cores_dominantes.items():
    print("Cor dominante das pétalas do arquivo '{}': {}".format(nome_arquivo, cor_dominante))

Cor dominante das pétalas do arquivo 'iris_versicolor_269': [58.27426886 74.43917424 38.30572622]
Cor dominante das pétalas do arquivo 'iris_setosa_65': [153.78710247 201.51442874 113.67520612]
Cor dominante das pétalas do arquivo 'iris_setosa_66': [161.82312457 156.87749484 213.57673778]
Cor dominante das pétalas do arquivo 'iris_virginica_71': [89.39608367 94.36315087 73.26183059]
Cor dominante das pétalas do arquivo 'iris_versicolor_233': [ 47.27237913 114.41168023  40.0576831 ]
Cor dominante das pétalas do arquivo 'iris_versicolor_223': [ 5.7122213  15.53323699  4.82586705]
Cor dominante das pétalas do arquivo 'iris_versicolor_240': [73.30989509 94.0392685  63.30989509]
Cor dominante das pétalas do arquivo 'iris_versicolor_261': [ 69.03417038 169.80944056  94.08773045]
Cor dominante das pétalas do arquivo 'iris_virginica_81': [25.146195   22.96833715 21.85577028]
Cor dominante das pétalas do arquivo 'iris_versicolor_256': [ 76.38770219 109.91591555  49.57253982]
Cor dominante das p

In [9]:
def extrair_formato_petalas(caminho_imagem):
    # Carregar a imagem em escala de cinza
    imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)

    # Aplicar um limiar para separar as pétalas do fundo
    _, imagem_limiar = cv2.threshold(imagem, 127, 255, cv2.THRESH_BINARY)

    # Encontrar os contornos na imagem
    contornos, _ = cv2.findContours(imagem_limiar, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calcular a área e o perímetro de cada contorno
    areas = [cv2.contourArea(contorno) for contorno in contornos]
    perimetros = [cv2.arcLength(contorno, True) for contorno in contornos]

    # Calcular a convexidade de cada contorno (a proporção entre a área do contorno e a área do seu casco convexo)
    convexidades = [cv2.contourArea(contorno) / cv2.contourArea(cv2.convexHull(contorno)) if cv2.contourArea(cv2.convexHull(contorno)) != 0 else 0 for contorno in contornos]

    return areas, perimetros, convexidades

In [10]:
def extrair_caracteristicas_pasta(caminho_pasta):
    # Obter uma lista de todos os arquivos na pasta
    arquivos = os.listdir(caminho_pasta)

    # Inicializar um dicionário para armazenar as características de cada arquivo
    caracteristicas_arquivos = {}

    # Para cada arquivo na pasta, extrair as características e adicionar ao dicionário
    for arquivo in arquivos:
        # Ignorar arquivos que não são imagens
        if not (arquivo.endswith('.png') or arquivo.endswith('.jpg')):
            continue

        # Extrair as características do arquivo
        areas, perimetros, convexidades = extrair_formato_petalas(os.path.join(caminho_pasta, arquivo))

        # Adicionar as características ao dicionário, usando o nome do arquivo sem a extensão como chave
        nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
        caracteristicas_arquivos[nome_arquivo_sem_extensao] = {"areas": areas, "perimetros":perimetros, "convexidades":convexidades}
    # Extrair o formato das pétalas

    return caracteristicas_arquivos

In [11]:
# # Exemplo de uso
# caracteristicas = extrair_caracteristicas('sample/iris/iris_setosa_1.jpg')

# print("Características da imagem:", caracteristicas)


# Exemplo de uso
caracteristicas_iris = extrair_caracteristicas_pasta('sample/iris/test')

for nome_arquivo, caracteristicas in caracteristicas_iris.items():
    print("Características do arquivo iris'{}': {}".format(nome_arquivo, caracteristicas))

Características do arquivo iris'iris_versicolor_269': {'areas': [0.0, 25.5, 0.0, 1.5, 0.0, 754.0, 0.0, 7.0, 0.0, 0.0, 0.0, 0.0, 0.0, 642.0, 2.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 1.5, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 40.5, 0.0, 0.0, 0.0, 0.0, 2.5, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 17.5, 78.0, 0.5, 2.0, 0.0, 2.0, 1.5, 2.0, 4.0, 0.0, 10.0, 2.5, 0.0, 0.0, 11.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5, 13180.5, 0.0, 0.0], 'perimetros': [2.0, 21.899494767189026, 0.0, 5.414213538169861, 0.0, 164.08326017856598, 2.0, 19.313708186149597, 0.0, 2.8284270763397217, 0.0, 2.0, 5.656854152679443, 133.74011433124542, 9.656854152679443, 0.0, 3.414213538169861, 0.0, 2.0, 2.0, 4.0, 7.414213538169861, 0.0, 2.0, 6.828427076339722, 8.485281229019165, 0.0, 4.0, 2.8284270763397217, 2.0, 18.485281229019165, 46.870057106018066, 2.0, 0.0, 0.0, 0.0, 11.071067690849304, 5.414213538169861, 2.8284270763397217, 2.0, 0.0, 0.0, 0.0, 0.0, 29.55634891986847, 75.2548331022

In [12]:
def reduzir_dimensao(dicionario_caracteristicas):
    dicionario_reduzido = {}

    for nome_arquivo, caracteristicas in dicionario_caracteristicas.items():
        caracteristicas_reduzidas = {chave: [np.mean(valores)] for chave, valores in caracteristicas.items()}
        dicionario_reduzido[nome_arquivo] = caracteristicas_reduzidas

    return dicionario_reduzido

caracteristicas_iris_reduzidas = reduzir_dimensao(caracteristicas_iris)

for nome_arquivo, caracteristicas in caracteristicas_iris_reduzidas.items():
    print("Características reduzidas do arquivo iris'{}': {}".format(nome_arquivo, caracteristicas))

Características reduzidas do arquivo iris'iris_versicolor_269': {'areas': [189.98717948717947], 'perimetros': [34.18008995973147], 'convexidades': [0.26005680607783016]}
Características reduzidas do arquivo iris'iris_setosa_65': {'areas': [105.41252955082743], 'perimetros': [13.720641083187527], 'convexidades': [0.1657472886742808]}
Características reduzidas do arquivo iris'iris_setosa_66': {'areas': [93.95029239766082], 'perimetros': [34.96794356379593], 'convexidades': [0.2848908170378267]}
Características reduzidas do arquivo iris'iris_virginica_71': {'areas': [105.82738095238095], 'perimetros': [36.73027951092947], 'convexidades': [0.37053598424843026]}
Características reduzidas do arquivo iris'iris_versicolor_233': {'areas': [241.08219178082192], 'perimetros': [63.463689114949474], 'convexidades': [0.2660257951428182]}
Características reduzidas do arquivo iris'iris_versicolor_223': {'areas': [48.9390243902439], 'perimetros': [24.37343276806963], 'convexidades': [0.2721249444396259

In [13]:
cores_dominantes_df = pd.DataFrame(cores_dominantes).T
cores_dominantes_df.columns = ['cor_r', 'cor_g', 'cor_b']
cores_dominantes_df = cores_dominantes_df.reset_index().rename(columns={'index':'arquivo_cor'})
cores_dominantes_df

,arquivo_cor,cor_r,cor_g,cor_b
0,iris_versicolor_269,58.274269,74.439174,38.305726
1,iris_setosa_65,153.787102,201.514429,113.675206
2,iris_setosa_66,161.823125,156.877495,213.576738
3,iris_virginica_71,89.396084,94.363151,73.261831
4,iris_versicolor_233,47.272379,114.411680,40.057683
...,...,...,...,...
79,iris_versicolor_257,22.624253,21.750100,18.368777
80,iris_versicolor_266,46.085549,93.296187,8.370761
81,iris_versicolor_244,69.179670,69.855678,43.278571
82,iris_setosa_58,104.779359,102.394306,153.818505


In [14]:
df = pd.DataFrame(caracteristicas_iris_reduzidas).T
df = df.reset_index().rename(columns={'index': 'nome_arquivo'})
df['classe'] = [classe.split('_')[1] for classe in df['nome_arquivo']]
df['areas'] = [area[0] for area in df['areas']]
df['perimetros'] = [perimetro[0] for perimetro in df['perimetros']]
df['convexidades'] = [convexidade[0] for convexidade in df['convexidades']]   
df

,nome_arquivo,areas,perimetros,convexidades,classe
0,iris_versicolor_269,189.987179,34.180090,0.260057,versicolor
1,iris_setosa_65,105.412530,13.720641,0.165747,setosa
2,iris_setosa_66,93.950292,34.967944,0.284891,setosa
3,iris_virginica_71,105.827381,36.730280,0.370536,virginica
4,iris_versicolor_233,241.082192,63.463689,0.266026,versicolor
...,...,...,...,...,...
79,iris_versicolor_257,34.597087,26.268231,0.242448,versicolor
80,iris_versicolor_266,118.652778,43.430001,0.458018,versicolor
81,iris_versicolor_244,13.294968,13.472129,0.141245,versicolor
82,iris_setosa_58,614.027778,56.791969,0.249676,setosa


In [15]:
df_carac = pd.merge(df, cores_dominantes_df, left_on='nome_arquivo', right_on='arquivo_cor')
df_carac.drop(columns=['arquivo_cor'], inplace=True)
df_carac

,nome_arquivo,areas,perimetros,convexidades,classe,cor_r,cor_g,cor_b
0,iris_versicolor_269,189.987179,34.180090,0.260057,versicolor,58.274269,74.439174,38.305726
1,iris_setosa_65,105.412530,13.720641,0.165747,setosa,153.787102,201.514429,113.675206
2,iris_setosa_66,93.950292,34.967944,0.284891,setosa,161.823125,156.877495,213.576738
3,iris_virginica_71,105.827381,36.730280,0.370536,virginica,89.396084,94.363151,73.261831
4,iris_versicolor_233,241.082192,63.463689,0.266026,versicolor,47.272379,114.411680,40.057683
...,...,...,...,...,...,...,...,...
79,iris_versicolor_257,34.597087,26.268231,0.242448,versicolor,22.624253,21.750100,18.368777
80,iris_versicolor_266,118.652778,43.430001,0.458018,versicolor,46.085549,93.296187,8.370761
81,iris_versicolor_244,13.294968,13.472129,0.141245,versicolor,69.179670,69.855678,43.278571
82,iris_setosa_58,614.027778,56.791969,0.249676,setosa,104.779359,102.394306,153.818505


In [ ]:
def normalizar_colunas(df):
    return df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
df_normalizado = normalizar_colunas(df_carac[['areas','perimetros','convexidades','cor_r','cor_g','cor_b']])
print(df_normalizado)
df_carac[['areas','perimetros','convexidades','cor_r','cor_g','cor_b']] = df_normalizado[['areas','perimetros','convexidades','cor_r','cor_g','cor_b']]
df_carac

In [16]:
df_carac['geral'] = [np.array([df_carac['areas'][i], df_carac['perimetros'][i], df_carac['convexidades'][i], df_carac['cor_r'][i], df_carac['cor_g'][i], df_carac['cor_b'][i]]) for i in range(len(df_carac))]
# df_carac['areas'][2]
df_carac.sort_values(by='nome_arquivo', inplace=True)
df_carac.reset_index(drop=True, inplace=True)
df_carac

,nome_arquivo,areas,perimetros,convexidades,classe,cor_r,cor_g,cor_b,geral
0,iris_setosa_55,171.927885,40.850386,0.242437,setosa,23.383403,86.727547,16.235557,"[171.9278846153846, 40.85038598111043, 0.24243..."
1,iris_setosa_56,116.768817,44.329692,0.279369,setosa,65.959971,113.949199,27.027802,"[116.76881720430107, 44.32969207404762, 0.2793..."
2,iris_setosa_57,1147.882353,76.525180,0.162317,setosa,135.790702,163.820702,110.795965,"[1147.8823529411766, 76.52517967714982, 0.1623..."
3,iris_setosa_58,614.027778,56.791969,0.249676,setosa,104.779359,102.394306,153.818505,"[614.0277777777778, 56.791968781087135, 0.2496..."
4,iris_setosa_59,119.841463,40.075949,0.304834,setosa,18.884928,41.735245,12.452150,"[119.84146341463415, 40.07594894781345, 0.3048..."
...,...,...,...,...,...,...,...,...,...
79,iris_virginica_81,1395.500000,128.123998,0.466382,virginica,25.146195,22.968337,21.855770,"[1395.5, 128.1239978969097, 0.4663822944027139..."
80,iris_virginica_82,4786.269231,126.481302,0.109638,virginica,176.984285,194.435165,153.138861,"[4786.2692307692305, 126.48130161028642, 0.109..."
81,iris_virginica_83,313.449495,53.691366,0.307275,virginica,126.280658,138.869680,34.538513,"[313.449494949495, 53.69136560324466, 0.307274..."
82,iris_virginica_84,44.822981,25.492963,0.218105,virginica,113.983687,131.368361,69.210926,"[44.82298136645963, 25.492962644707344, 0.2181..."


In [17]:
df_carac.to_csv('sample/iris/iris_test.csv', index=False, sep=',', header=True)

In [ ]:
def early_fusion(caracteristicas, caracteristicas_outras):
    # Inicializar um dicionário para armazenar os vetores fusionados
    dicionario_fusion = {}

    # Para cada arquivo no dicionário de características, adicionar o vetor fusionado ao dicionario_fusion
    for nome_arquivo, caracteristica in caracteristicas.items():
        # Ignorar arquivos que não estão no dicionario_cores
        if nome_arquivo not in caracteristicas_outras:
            continue

        # Obter a cor correspondente
        outra = caracteristicas_outras[nome_arquivo]

        # Realizar a fusão precoce concatenando as características e a cor
        vetor_fusion = caracteristica + outra

        # Adicionar o vetor fusionado ao dicionario_fusion
        dicionario_fusion[nome_arquivo] = vetor_fusion

    return dicionario_fusion

In [ ]:
# for nome_arquivo, caracteristica in caracteristicas_iris_reduzidas.items():
    # print("Características reduzidas do arquivo iris'{}': {}".format(nome_arquivo, caracteristicas))
for nome_arquivo, caracteristica in caracteristicas_iris_reduzidas.items():
    nova_caracteristica = []
    nova_caracteristica.append(caracteristica['areas'][0])
    nova_caracteristica.append(caracteristica['perimetros'][0])
    nova_caracteristica.append(caracteristica['convexidades'][0])
    caracteristica_iris = {nome_arquivo: np.array(nova_caracteristica)}
    print("Características early fusion do arquivo iris'{}': {}".format(nome_arquivo, caracteristica_iris[nome_arquivo]))

In [ ]:
caracteristica_iris

In [ ]:
fusion_areas_diametros = early_fusion(caracteristicas_iris_reduzidas, images_vectorized)

In [ ]:
caracteristica_iris_cores = {}
for nome_arquivo_cor, caracteristica_cor in cores_dominantes.items():
    for nome_arquivo, caracteristica in caracteristica_iris.items():
        if nome_arquivo_cor == nome_arquivo:
            print(True)
            array_caracteristica = np.append(caracteristica, caracteristica_cor)
            caracteristica_iris_cores[nome_arquivo] = array_caracteristica

In [ ]:
caracteristica_iris_cores

In [ ]:
for nome_arquivo_cor, caracteristica_cor in cores_dominantes.items(): 
    print(nome_arquivo_cor, caracteristica_cor)

In [ ]:
caracteristica_iris_cores

In [ ]:
cores_dominantes

### Gemini Tutorial

In [ ]:
import pandas as pd

# Carregar o dataset
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00456/Multimodal+Damage+Identification+for+Humanitarian+Computing.csv")

# Separar as features em imagens, textos e informações tabulares
images = data["image_url"]
texts = data["text"]
tabular_data = data.drop(["image_url", "text"], axis=1)


In [ ]:
# Carregar imagens
images = tf.keras.utils.image_dataset_from_directory(
    directory="data/images",
    label_mode="categorical",
    image_size=(224, 224),
    batch_size=32
)

# Carregar áudio
audio = tf.keras.utils.audio_dataset_from_directory(
    directory="data/audio",
    label_mode="categorical",
    batch_size=32
)
